In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Tải lại mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/my_model')
tokenizer = AutoTokenizer.from_pretrained('roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
# Tải dataset
dataset = load_dataset("BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong")

# 2. Khởi tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# 3. Tiền xử lý câu hỏi và câu trả lời
answers = dataset['train']['answer']  # Danh sách câu trả lời
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(answers)  # Mã hóa câu trả lời thành nhãn

README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/72.9k [00:00<?, ?B/s]

(…)_test_tdt_dataset-00000-of-00001.parquet:   0%|          | 0.00/18.0k [00:00<?, ?B/s]

(…).medium_index_TDT-00000-of-00001.parquet:   0%|          | 0.00/65.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/215 [00:00<?, ? examples/s]

Generating TEST.basic_test_tdt_dataset split:   0%|          | 0/20 [00:00<?, ? examples/s]

Generating INDEX.medium_index_TDT split:   0%|          | 0/144 [00:00<?, ? examples/s]

In [ ]:
# Kiểm tra kết quả
print(answers[0])
print(dataset)

Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.

Nếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.
DatasetDict({
    train: Dataset({
        features: ['question', 'answer', 'url', 'group', 'doc_id', 'metadata'],
        num_rows: 215
    })
    TEST.basic_test_tdt_dataset: Dataset({
        features: ['question', 'answer', 'url', 'group', 'doc_

In [ ]:
# 4. Tiền xử lý dữ liệu
def preprocess_function(examples):
    inputs = tokenizer(examples['question'], truncation=True, padding='max_length', max_length=512)
    inputs['labels'] = labels[:len(examples['question'])]  # Thêm nhãn
    return inputs

# Tiền xử lý dữ liệu
tokenized_datasets = dataset.map(preprocess_function, batched=True)
num_labels = len(set(labels))  # Số lượng nhãn duy nhất

Map:   0%|          | 0/144 [00:00<?, ? examples/s]

In [ ]:
# Sử dụng mô hình phân loại
model = AutoModelForSequenceClassification.from_pretrained("roberta-base", num_labels=len(set(labels)))
# Huấn luyện mô hình
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",  # Lưu checkpoint tại mỗi epoch
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=100,
    report_to="none",
    resume_from_checkpoint='./results/checkpoint-<epoch_number>',
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['train'],
)

trainer.train()

In [ ]:
import torch
# Giả sử bạn đã khởi tạo mô hình và chuyển nó đến GPU
model.to('cuda')

# Hàm dự đoán
def predict(question):
    # Tiền xử lý câu hỏi
    inputs = tokenizer(question, return_tensors='pt').to('cuda')  # Chuyển tensor đến GPU
    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy logits và tính xác suất
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)

    # Lấy chỉ số của câu trả lời có xác suất cao nhất
    answer_index = probabilities.argmax(dim=-1)

    # Lấy xác suất của câu trả lời
    answer_probability = probabilities.max(dim=-1).values.item()

    return answer_index, answer_probability

In [ ]:
question = "Thủ tục và cách thức gia hạn đóng học phí như thế nào?"
answer = predict(question)

In [ ]:
question = "Thủ tục và cách thức gia hạn đóng học phí như thế nào?"
answer_index, answer_probability = predict(question)

# Giải mã kết quả
decoded_answer = label_encoder.inverse_transform(answer_index.cpu().numpy())
print(f"Câu trả lời: {decoded_answer}, Độ tin cậy: {answer_probability:.2f}")

Câu trả lời: ['Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.'], Độ tin cậy: 0.02


In [ ]:
model.save_pretrained('./my_model')
tokenizer.save_pretrained('./my_model')

# **Muốn test thì chạy từ đây**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets

In [6]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
# Tải dataset
dataset = load_dataset("BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong")

from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Tải lại mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/my_model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/my_model')

# 2. Khởi tạo tokenizer
tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# 3. Tiền xử lý câu hỏi và câu trả lời
answers = dataset['train']['answer']  # Danh sách câu trả lời
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(answers)
print(labels)


[108 188 107 194 180 196  87  31  25  30  27 189  54 154  50 117  49 116
 193  57 103 139  58  10  88 140 157  60 181  59 146 142 207  48 144  47
  98  96 145 141  99   5   0  52  92   8   3  18 168 201 170 200 172 208
 186 187 190 173 199 198  21  20 191 176  71 205  73 183  78 192 152  26
 143   7   9 209  53  93  62  79  68 175  17 164 174 110 203  19  77  94
  95  86  15 149  67  70 106 101 153  51 163  75 195 120  16  65 118 178
 171 119  91 169  89  11   4 210  14   1   2 100 212  63  80 211  74 147
 160 155 158 150 151 156  64 165 206  55  90  10  66 213  12 102  76  97
  22  69 166 204   6 148 105  13  61 167  56 179  24 182 184 162 161 177
 159  23 202 109  28  29 197 185  42 131  81  44 133  83  41 125  82  36
 132  85  33 130  84  37 129 104  39 135 121  35 137 122  40 138 123  38
 128  72  43 124 113  46 136 114  45 127 112  32 126 111  34 134 115]


In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import json
import torch
from sklearn.preprocessing import LabelEncoder

# Lấy nhãn từ tệp JSON
# Tải nhãn từ config.json
with open('/content/drive/MyDrive/my_model/config.json', 'r') as f:
    config = json.load(f)

#id2label = config['id2label']
#label2id = config['label2id']

# Giả sử bạn đã khởi tạo mô hình và chuyển nó đến GPU
#model.to('cuda')
model.to('cpu')

# Hàm dự đoán
def predict(question):
    # Tiền xử lý câu hỏi
    inputs = tokenizer(question, return_tensors='pt').to('cpu')
    with torch.no_grad():
        outputs = model(**inputs)

    # Lấy logits và tính xác suất
    logits = outputs.logits
    probabilities = torch.softmax(logits, dim=-1)

    # Lấy chỉ số của câu trả lời có xác suất cao nhất
    answer_index = probabilities.argmax(dim=-1)

    # Lấy xác suất của câu trả lời
    answer_probability = probabilities.max(dim=-1).values.item()

    return answer_index, answer_probability, probabilities

# Ví dụ kiểm tra
question = "Thủ tục và cách thức gia hạn đóng học phí như thế nào?"
answer_index, answer_probability, probabilities = predict(question)

print(f"Index của câu trả lời: {answer_index}, Xác suất: {answer_probability:.2f}")

# Giải mã kết quả
decoded_answer = label_encoder.inverse_transform(answer_index.cpu().numpy())
print(f"Câu trả lời: {decoded_answer}, Độ tin cậy: {answer_probability:.2f}")

Index của câu trả lời: tensor([108]), Xác suất: 0.02
Câu trả lời: ['Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời gian đóng học phí, đính kèm minh chứng và nộp về cho phòng Công tác tác Học sinh - Sinh viên (A.0003) trong thời gian đóng học phí.'], Độ tin cậy: 0.02


**Chay code mở fastAPI từ đây**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install fastapi uvicorn pyngrok nest_asyncio
!pip install datasets

In [13]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from sklearn.preprocessing import LabelEncoder
# Tải dataset
dataset = load_dataset("BroDeadlines/QA.TDT.FQA_tu_van_hoc_duong")

from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Tải lại mô hình và tokenizer
model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/my_model')
tokenizer = AutoTokenizer.from_pretrained('/content/drive/MyDrive/my_model')

model.to('cpu')

# 2. Khởi tạo tokenizer
#tokenizer = AutoTokenizer.from_pretrained("roberta-base")

# 3. Tiền xử lý câu hỏi và câu trả lời
answers = dataset['train']['answer']  # Danh sách câu trả lời
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(answers)
print(labels)

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-7' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py", l

[108 188 107 194 180 196  87  31  25  30  27 189  54 154  50 117  49 116
 193  57 103 139  58  10  88 140 157  60 181  59 146 142 207  48 144  47
  98  96 145 141  99   5   0  52  92   8   3  18 168 201 170 200 172 208
 186 187 190 173 199 198  21  20 191 176  71 205  73 183  78 192 152  26
 143   7   9 209  53  93  62  79  68 175  17 164 174 110 203  19  77  94
  95  86  15 149  67  70 106 101 153  51 163  75 195 120  16  65 118 178
 171 119  91 169  89  11   4 210  14   1   2 100 212  63  80 211  74 147
 160 155 158 150 151 156  64 165 206  55  90  10  66 213  12 102  76  97
  22  69 166 204   6 148 105  13  61 167  56 179  24 182 184 162 161 177
 159  23 202 109  28  29 197 185  42 131  81  44 133  83  41 125  82  36
 132  85  33 130  84  37 129 104  39 135 121  35 137 122  40 138 123  38
 128  72  43 124 113  46 136 114  45 127 112  32 126 111  34 134 115]


In [27]:
from fastapi import FastAPI, File, UploadFile
from fastapi.responses import JSONResponse
from fastapi.middleware.cors import CORSMiddleware
from tensorflow.keras.models import load_model
from fastapi import FastAPI, HTTPException
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import numpy as np
import io
import torch
from pydantic import BaseModel
from PIL import Image
import uvicorn
from pyngrok import ngrok
import nest_asyncio
import json

# Áp dụng nest_asyncio để hỗ trợ vòng lặp sự kiện trong Colab
nest_asyncio.apply()

# Khởi tạo ứng dụng FastAPI
app = FastAPI()
# Cấu hình CORS
origins = [
    "http://127.0.0.1:5500",  # Miền phát triển của bạn
    "http://localhost:5500",
    "https://<your-ngrok-url>.ngrok.io"  # URL Ngrok của bạn
]
# Thêm CORS middleware để cho phép các yêu cầu từ bất kỳ nguồn gốc nào
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # Cho phép tất cả các nguồn gốc
    allow_credentials=True,
    allow_methods=["*"],  # Cho phép tất cả các phương thức HTTP
    allow_headers=["*"],  # Cho phép tất cả các tiêu đề
)

# Định nghĩa mô hình đầu vào
class Question(BaseModel):
    question: str

# Hàm dự đoán
def predict(question):
    try:
        print(f"Processing question: {question}")  # Ghi log câu hỏi
        inputs = tokenizer(question, return_tensors='pt').to('cpu')
        with torch.no_grad():
            outputs = model(**inputs)

        logits = outputs.logits
        probabilities = torch.softmax(logits, dim=-1)
        answer_index = probabilities.argmax(dim=-1)
        answer_probability = probabilities.max(dim=-1).values.item()

        print(f"Predicted index: {answer_index}, Probability: {answer_probability}")  # Ghi log kết quả
        return answer_index.item(), answer_probability,
    except Exception as e:
        print(f"Error in predict: {str(e)}")  # Ghi log lỗi chi tiết
        raise
# Endpoint nhận ảnh và trả về kết quả dự đoán
@app.post("/predict/")
async def get_prediction(question: Question):
    try:
        print(f"Received question: {question.question}")
        answer_index, answer_probability = predict(question.question)
        print(f"Predicted index: {answer_index}, Probability: {answer_probability}")

        if isinstance(answer_index, torch.Tensor):
            answer_index = answer_index.item()

        decoded_answer = label_encoder.inverse_transform([answer_index])
        print(f"Decoded answer: {decoded_answer}")

        # Tách nhỏ đáp án nếu cần
        answer_text = decoded_answer[0] if hasattr(decoded_answer, '__iter__') else decoded_answer
        if len(answer_text) > 750:
            # Chia nhỏ đáp án thành các đoạn 750 ký tự
            chunks = [answer_text[i:i + 750] for i in range(0, len(answer_text), 750)]
        else:
            chunks = [answer_text]

        return {"answer": chunks}
    except Exception as e:
        print(f"Error: {str(e)}")
        raise HTTPException(status_code=500, detail=str(e))

# Thêm authtoken của ngrok
ngrok.set_auth_token("2pfTSXoN32WabYzXtnJ0iDzHW6W_5VetNxdLwi471Rkjjst3B")

# Khởi chạy ngrok và FastAPI
def start_ngrok():
    public_url = ngrok.connect(8000)
    print(f"Lấy link này để dổi: NgrokTunnel + /predict/")
    print(f"Public URL: {public_url}")

def start_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Chạy cả hai trong Colab
if __name__ == "__main__":
    start_ngrok()
    start_fastapi()

#Thủ tục và cách thức gia hạn đóng học phí như thế nào?

ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-105' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/usr/lib/python3.10/asyncio/tasks.py",

Lấy link này để dổi: NgrokTunnel + /predict/
Public URL: NgrokTunnel: "https://a3d2-34-127-120-22.ngrok-free.app" -> "http://localhost:8000"


INFO:     Started server process [288]
INFO:     Waiting for application startup.
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-67' coro=<Server.serve() done, defined at /usr/local/lib/python3.10/dist-packages/uvicorn/server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/main.py", line 579, in run
    server.run()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.10/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run
    self._contex

INFO:     2405:4803:b187:9540:bd82:cb67:e078:65b9:0 - "OPTIONS /predict/ HTTP/1.1" 200 OK
Received question: Thủ tục và cách thức gia hạn đóng học phí như thế nào?
Processing question: Thủ tục và cách thức gia hạn đóng học phí như thế nào?
Predicted index: tensor([108]), Probability: 0.018943721428513527
Predicted index: 108, Probability: 0.018943721428513527
Decoded answer: ['Trong thời gian đóng học phí theo thông báo của phòng Tài chính, sinh viên thuộc diện khó khăn (có minh chứng) có thể nộp đơn xin gia hạn học phí trực tuyến bằng cách vào Hệ thống thông tin sinh viên, phân hệ Nộp đơn trực tuyến, chọn học kỳ muốn gia hạn học phí, chọn loại đơn cần nộp và tiến hành điền các thông tin cần thiết. Sinh viên theo dõi kết quả trả lời đơn để biết đơn có được cho gia hạn hay không. Việc nộp đơn trực tuyến chỉ áp dụng với đợt đóng học phí đầu tiên của học kỳ chính.\n\nNếu sinh viên có nguyện vọng gia hạn thời gian đóng học phí ở các đợt đóng tiếp theo, sinh viên có thể nộp đơn gia hạn thời

INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [288]
